In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.auto import tqdm
import os
import bar_chart_race as bcr

In [4]:
df_games = pd.read_csv('Games.csv')
df_games['home_team'] = df_games['hometeamCity'].str.strip() + ' ' + df_games['hometeamName'].str.strip()
df_games['away_team'] = df_games['awayteamCity'].str.strip() + ' ' + df_games['awayteamName'].str.strip()
df_games['gameDate'] = pd.to_datetime(df_games['gameDate'])
df_games['game_date'] = df_games['gameDate'].dt.date
df_games=df_games[['game_date', 'home_team', 'away_team', 'homeScore', 'awayScore', 'gameType']]
df_games

C:\Users\llama\AppData\Local\Temp\ipykernel_15996\921228088.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_games = pd.read_csv('Games.csv')


,game_date,home_team,away_team,homeScore,awayScore,gameType
0,2025-06-22,Oklahoma City Thunder,Indiana Pacers,103,91,Playoffs
1,2025-06-19,Indiana Pacers,Oklahoma City Thunder,108,91,Playoffs
2,2025-06-16,Oklahoma City Thunder,Indiana Pacers,120,109,Playoffs
3,2025-06-13,Indiana Pacers,Oklahoma City Thunder,104,111,Playoffs
4,2025-06-11,Indiana Pacers,Oklahoma City Thunder,116,107,Playoffs
...,...,...,...,...,...,...
71872,1946-12-08,New York Knicks,Boston Celtics,62,44,Regular Season
71873,1946-12-07,Boston Celtics,New York Knicks,65,90,Regular Season
71874,1946-12-05,Philadelphia Warriors,New York Knicks,62,51,Regular Season
71875,1946-11-30,New York Knicks,Philadelphia Warriors,64,60,Regular Season


In [5]:
df_players = pd.read_csv('PlayerStatistics.csv')
df_players['gameDate'] = pd.to_datetime(df_players['gameDate'])
df_players['game_date'] = df_players['gameDate'].dt.date
sorted_dates = sorted(df_players['game_date'].unique())
df_players['fullName'] = df_players['firstName'].str.strip() + ' ' + df_players['lastName'].str.strip()
df_players['player_team'] = df_players['playerteamCity'].str.strip() + ' ' + df_players['playerteamName'].str.strip()
df_players['opponent_team'] = df_players['opponentteamCity'].str.strip() + ' ' + df_players['opponentteamName'].str.strip()
df_players['numMinutes'] = df_players['numMinutes'].replace([np.inf, -np.inf], np.nan).fillna(0.0)
minutes = df_players['numMinutes'].astype(int)
seconds = ((df_players['numMinutes'] - minutes) * 100).round()
df_players['minutes']= minutes+seconds/60
players = df_players['fullName'].unique()
df_players = df_players[['game_date','fullName','player_team', 'opponent_team', 'minutes']]
df_players

C:\Users\llama\AppData\Local\Temp\ipykernel_15996\3596957360.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_players = pd.read_csv('PlayerStatistics.csv')


,game_date,fullName,player_team,opponent_team,minutes
0,2025-06-22,James Johnson,Indiana Pacers,Oklahoma City Thunder,0.000000
1,2025-06-22,T.J. McConnell,Indiana Pacers,Oklahoma City Thunder,28.083333
2,2025-06-22,Myles Turner,Indiana Pacers,Oklahoma City Thunder,23.816667
3,2025-06-22,Pascal Siakam,Indiana Pacers,Oklahoma City Thunder,36.916667
4,2025-06-22,Alex Caruso,Oklahoma City Thunder,Indiana Pacers,32.416667
...,...,...,...,...,...
1627433,1946-11-26,Fred Sheffield,Philadelphia Warriors,Boston Celtics,0.000000
1627434,1946-11-26,Connie Simmons,Boston Celtics,Philadelphia Warriors,0.000000
1627435,1946-11-26,Johnny Simmons,Boston Celtics,Philadelphia Warriors,0.000000
1627436,1946-11-26,Virgil Vaughn,Boston Celtics,Philadelphia Warriors,0.000000


In [ ]:
# Function to compute team average ELO based on minutes played
def calculate_team_ELO(players_rows, last_ELO):
    total_minutes = players_rows['minutes'].sum()
    num_players = len(players_rows)
    # Handle invalid or missing minutes
    assert num_players > 0, f"{players_rows}"

    if total_minutes <= 0 or num_players < 5:
        # Default total minutes (full game for 5 players)
        total_minutes = 48 * 5
        minutes_per_player = total_minutes / num_players
        weighted_sum = sum(last_ELO[row['fullName']] * minutes_per_player for _, row in players_rows.iterrows())
        return weighted_sum / total_minutes, total_minutes, minutes_per_player
    else:
        weighted_sum = sum(last_ELO[row['fullName']] * row['minutes'] for _, row in players_rows.iterrows())
        return weighted_sum / total_minutes, total_minutes, None


In [ ]:
save_dir = 'data'
os.makedirs(save_dir, exist_ok=True)

# --- LOAD CHECKPOINT IF EXISTS ---
saved_files = [f for f in os.listdir(save_dir) if f.startswith("elo_history_until_")]
if saved_files:
    # Encuentra la última fecha guardada
    latest_file = sorted(saved_files)[-1]
    last_saved_date = latest_file.split("_until_")[1].split(".csv")[0]

    print(f"🔁 Resuming from {last_saved_date}...")

    # Load ELO history
    elo_history_df = pd.read_csv(os.path.join(save_dir, latest_file))
    ELO = {player: elo_history_df[player].dropna().tolist() for player in elo_history_df.columns}
    length = len(elo_history_df)

    # Load last ELO
    last_elo_df = pd.read_csv(os.path.join(save_dir, f"last_elo_until_{last_saved_date}.csv"))
    last_ELO = dict(zip(last_elo_df['player'], last_elo_df['last_ELO']))

    # Load highest ELO
    highest_elo_df = pd.read_csv(os.path.join(save_dir, f"highest_elo_until_{last_saved_date}.csv"))
    highest_ELO = dict(zip(highest_elo_df['player'], highest_elo_df['highest_ELO']))

    # Load players
    players = list(last_ELO.keys())

    # Filter later dates
    sorted_dates_filtered = [d for d in sorted_dates if d > datetime.datetime.strptime(last_saved_date, "%Y-%m-%d").date()]
    df_players_filtered=df_players[df_players['game_date']> datetime.datetime.strptime(last_saved_date, "%Y-%m-%d").date()].copy()
    df_games_filtered=df_games[df_games['game_date']> datetime.datetime.strptime(last_saved_date, "%Y-%m-%d").date()].copy()
else:
    print("🚀 Initializing from scratch...")

    # Initialize player ELO structures
    ELO = {player: [1000] for player in players} # Full ELO history per player
    last_ELO = {player: 1000 for player in players} # Latest ELO per player
    highest_ELO = {player: 1000 for player in players} # Highest ELO achieved per player
    length = 1 # Track length of ELO history
    df_players_filtered=df_players.copy()
    df_games_filtered=df_games.copy()

df_games_filtered.set_index('game_date', inplace=True)
df_players_filtered.set_index('game_date', inplace=True)

display(df_players_filtered)
display(df_games_filtered)

# Main loop through all game dates
for game_date in tqdm(sorted_dates_filtered):
    daily_games = df_games_filtered.loc[[game_date]]
    players_today = df_players_filtered.loc[[game_date]]

    for game in daily_games.itertuples(index=False):
        home_team = game.home_team
        away_team = game.away_team
        team_players = {
            home_team: players_today[players_today['player_team'] == home_team],
            away_team: players_today[players_today['player_team'] == away_team]
        }
        team_ELO = {}
        team_time = {}
        default_minutes = {}

        # Calculate each team's average ELO and total minutes played
        for team in [home_team, away_team]:
            players_rows = team_players[team]
            team_ELO[team], team_time[team], default_minutes[team] = calculate_team_ELO(players_rows, last_ELO)

        # Calculate expected win probability using ELO difference
        Q_home = 1 / (1 + 10 ** ((team_ELO[away_team] - team_ELO[home_team]) / 400))
        Q_away = 1 - Q_home
        team_Q = {home_team: Q_home, away_team: Q_away}

        # Determine K factor (higher for playoff games)
        K = 32 if game.gameType == 'Playoffs' else 16

        # Update ELO ratings for each player based on game result
        for team in [home_team, away_team]:
            won = (team == home_team and game.homeScore > game.awayScore) or \
                  (team == away_team and game.awayScore > game.homeScore)
            S = 1 if won else 0
            E = team_Q[team]

            players_rows = team_players[team]

            for row in players_rows.itertuples(index=False):
                player = row.fullName
                current_ELO = last_ELO[player]
                # Use default minutes if available
                minutes_played = row.minutes if default_minutes[team] is None else default_minutes[team]

                # Update ELO based on player's minutes and game outcome (per 36 min)
                delta =  (minutes_played / 36) * K * (S - E)
                # print(f"Player: {player}. Minutes played {minutes_played}. Total time {team_time[team]} Delta {delta}")
                new_ELO = current_ELO + delta

                # Save new ELO
                ELO[player].append(new_ELO)
                last_ELO[player] = new_ELO
                highest_ELO[player] = max(highest_ELO[player], new_ELO)

    df_games_filtered.drop(index=game_date, inplace=True, errors='ignore')
    df_players_filtered.drop(index=game_date, inplace=True, errors='ignore')

    # Ensure every player has the same number of ELO entries (carry forward)
    length += 1
    for player in ELO:
      if len(ELO[player]) < length:
          ELO[player].append(last_ELO[player])

    # Show progress every 100 dates
    if length % 100 == 0:
      print(f"\n📅 Date: {game_date}")

      # Top 10 sorted lists
      top_highest = sorted(highest_ELO.items(), key=lambda x: x[1], reverse=True)[:10]
      top_current = sorted(last_ELO.items(), key=lambda x: x[1], reverse=True)[:10]

      # Header with fixed-width columns
      name_width = 20  # Adjust as needed
      print(
          f"{'🏆 Highest Player ELO 🏆':<{name_width}}        "
          f"{'🔥 Current Player ELO 🔥':<{name_width}}"
      )

      # Rows
      for (high_name, high_elo), (curr_name, curr_elo) in zip(top_highest, top_current):
          print(
              f"{high_name:<{name_width}} {high_elo:>7.2f}    "
              f"{curr_name:<{name_width}} {curr_elo:>7.2f}"
          )

    # Save progress every 500 dates
    if length % 1000 == 0:
        print(f"\n💾 Saving progress at date {game_date}...")

        # Save full ELO history
        elo_history_df = pd.DataFrame.from_dict(ELO, orient='index').transpose()
        elo_history_df.to_csv(f"elo_history_until_{game_date}.csv", index=False)

        # Save last ELO per player
        last_elo_df = pd.DataFrame(list(last_ELO.items()), columns=['player', 'last_ELO'])
        last_elo_df.to_csv(f"last_elo_until_{game_date}.csv", index=False)

        # Save highest ELO per player
        highest_elo_df = pd.DataFrame(list(highest_ELO.items()), columns=['player', 'highest_ELO'])
        highest_elo_df.to_csv(f"highest_elo_until_{game_date}.csv", index=False)

🔁 Resuming from 2024-05-23...


,fullName,player_team,opponent_team,minutes
game_date,,,,
2025-06-22,James Johnson,Indiana Pacers,Oklahoma City Thunder,0.000000
2025-06-22,T.J. McConnell,Indiana Pacers,Oklahoma City Thunder,28.083333
2025-06-22,Myles Turner,Indiana Pacers,Oklahoma City Thunder,23.816667
2025-06-22,Pascal Siakam,Indiana Pacers,Oklahoma City Thunder,36.916667
2025-06-22,Alex Caruso,Oklahoma City Thunder,Indiana Pacers,32.416667
...,...,...,...,...
2024-05-24,Jaden Hardy,Dallas Mavericks,Minnesota Timberwolves,11.000000
2024-05-24,Wendell Moore Jr.,Minnesota Timberwolves,Dallas Mavericks,0.000000
2024-05-24,Leonard Miller,Minnesota Timberwolves,Dallas Mavericks,0.000000


,home_team,away_team,homeScore,awayScore,gameType
game_date,,,,,
2025-06-22,Oklahoma City Thunder,Indiana Pacers,103,91,Playoffs
2025-06-19,Indiana Pacers,Oklahoma City Thunder,108,91,Playoffs
2025-06-16,Oklahoma City Thunder,Indiana Pacers,120,109,Playoffs
2025-06-13,Indiana Pacers,Oklahoma City Thunder,104,111,Playoffs
2025-06-11,Indiana Pacers,Oklahoma City Thunder,116,107,Playoffs
...,...,...,...,...,...
2024-05-28,Dallas Mavericks,Minnesota Timberwolves,100,105,Playoffs
2024-05-27,Indiana Pacers,Boston Celtics,102,105,Playoffs
2024-05-26,Dallas Mavericks,Minnesota Timberwolves,116,107,Playoffs


  0%|          | 0/239 [00:00<?, ?it/s]


📅 Date: 2025-01-07
🏆 Highest Player ELO 🏆        🔥 Current Player ELO 🔥
LeBron James         2142.04    Vlade Divac          1799.02
Michael Jordan       1926.82    Tim Duncan           1742.88
Chris Paul           1910.37    Michael Jordan       1730.55
Shaquille O'Neal     1905.90    Chris Paul           1702.93
Tim Duncan           1873.68    David Robinson       1699.53
Vlade Divac          1869.99    Kareem Abdul-Jabbar  1682.94
Dirk Nowitzki        1807.59    LeBron James         1650.63
Steve Nash           1764.44    Jrue Holiday         1629.54
Stephen Curry        1758.68    Larry Bird           1612.00
Kareem Abdul-Jabbar  1735.33    Magic Johnson        1601.40

📅 Date: 2025-04-26
🏆 Highest Player ELO 🏆        🔥 Current Player ELO 🔥
LeBron James         2142.04    Vlade Divac          1799.02
Michael Jordan       1926.82    Tim Duncan           1742.88
Chris Paul           1910.37    Michael Jordan       1730.55
Shaquille O'Neal     1905.90    David Robinson       1699.53


In [ ]:
elo_history_df = pd.DataFrame.from_dict(ELO, orient='index').transpose()
last_elo_df = pd.DataFrame(list(last_ELO.items()), columns=['fullName', 'last_ELO'])
highest_elo_df = pd.DataFrame(list(highest_ELO.items()), columns=['fullName', 'highest_ELO'])

In [ ]:
print(f"\n💾 Saving progress at date {game_date}...")

# Save full ELO history
elo_history_df.to_csv(f"elo_history_until_{game_date}.csv", index=False)

# Save last ELO per player
last_elo_df.to_csv(f"last_elo_until_{game_date}.csv", index=False)

# Save highest ELO per player
highest_elo_df.to_csv(f"highest_elo_until_{game_date}.csv", index=False)


💾 Saving progress at date 2025-06-22...


In [ ]:
elo_history_df['game_date']=[datetime.date(1946,11,25)]+sorted_dates
elo_history_df.set_index('game_date', inplace=True)
elo_history_df

,James Johnson,T.J. McConnell,Myles Turner,Pascal Siakam,Alex Caruso,Isaiah Hartenstein,Tony Bradley,Thomas Bryant,Shai Gilgeous-Alexander,Kenrich Williams,...,Bob Mullens,Hank Rosenstein,Moe Becker,Michael Wallace,Hal Crisler,Fred Sheffield,Mel Hirsch,Tony Kappen,Don Eliason,Virgil Vaughn
game_date,,,,,,,,,,,,,,,,,,,,,
1946-11-25,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1946-11-26,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,995.151515,1000.000000,1005.333333,995.151515,995.151515,1000.000000,995.151515
1946-11-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1006.016885,1000.000000,995.151515,1000.000000,998.564338,995.151515,995.151515,1000.000000,995.151515
1946-12-05,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.599290,1000.000000,995.151515,1000.000000,1003.489424,995.151515,995.151515,1000.000000,995.151515
1946-12-07,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1004.381853,1004.981143,1000.000000,990.769662,1000.000000,1003.489424,990.769662,990.769662,995.618147,995.151515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-11,960.550887,1203.930813,1384.659858,1305.570193,1172.986433,1299.905698,1005.946523,947.123289,1493.799478,1074.030994,...,997.959159,1004.172695,999.769542,985.414685,1005.054648,998.434776,985.645143,985.645143,995.618147,995.151515
2025-06-13,960.550887,1196.329495,1371.765145,1290.741626,1185.859972,1308.897414,1005.946523,947.123289,1510.801867,1078.809166,...,997.959159,1004.172695,999.769542,985.414685,1005.054648,998.434776,985.645143,985.645143,995.618147,995.151515
2025-06-16,960.550887,1188.185995,1362.614519,1278.408980,1196.728271,1316.626943,1001.399015,946.437456,1524.870723,1082.788236,...,997.959159,1004.172695,999.769542,985.414685,1005.054648,998.434776,985.645143,985.645143,995.618147,995.151515


# ELO rating of active NBA players over time

In [ ]:
df_unpivot = pd.melt(
    elo_history_df.reset_index(),
    id_vars=['game_date'],
    var_name='fullName',
    value_name='ELO'
)
df_unpivot

,game_date,fullName,ELO
0,1946-11-25,James Johnson,1000.000000
1,1946-11-26,James Johnson,1000.000000
2,1946-11-30,James Johnson,1000.000000
3,1946-12-05,James Johnson,1000.000000
4,1946-12-07,James Johnson,1000.000000
...,...,...,...
76292557,2025-06-11,Virgil Vaughn,995.151515
76292558,2025-06-13,Virgil Vaughn,995.151515
76292559,2025-06-16,Virgil Vaughn,995.151515
76292560,2025-06-19,Virgil Vaughn,995.151515


In [ ]:
df_career_length = df_players.groupby('fullName')['game_date'].agg(['min', 'max']).reset_index()
df_career_length

,fullName,min,max
0,A.C. Green,1985-10-26,2001-04-27
1,A.J. Bramlett,1999-11-30,2000-01-04
2,A.J. English,1990-11-02,1992-04-18
3,A.J. Guyton,2000-10-31,2003-10-07
4,A.J. Lawson,2021-10-04,2025-04-13
...,...,...,...
5353,Zoran Planinic,2003-04-19,2006-05-16
5354,Zydrunas Ilgauskas,1997-10-31,2011-05-26
5355,Zylan Cheatham,2019-10-07,2022-10-14
5356,Zyon Pullin,2024-10-08,2025-03-25


In [ ]:
df_filtered = pd.merge(
    highest_elo_df[highest_elo_df['highest_ELO']>1300],
    df_unpivot,
    on='fullName',
    how='left'
)
df_filtered


,fullName,highest_ELO,game_date,ELO
0,Myles Turner,1384.659858,1946-11-25,1000.000000
1,Myles Turner,1384.659858,1946-11-26,1000.000000
2,Myles Turner,1384.659858,1946-11-30,1000.000000
3,Myles Turner,1384.659858,1946-12-05,1000.000000
4,Myles Turner,1384.659858,1946-12-07,1000.000000
...,...,...,...,...
4086588,George Mikan,1304.147486,2025-06-11,1282.795065
4086589,George Mikan,1304.147486,2025-06-13,1282.795065
4086590,George Mikan,1304.147486,2025-06-16,1282.795065
4086591,George Mikan,1304.147486,2025-06-19,1282.795065


In [ ]:

df_active_filtered = pd.merge(
    df_filtered,
    df_career_length,
    on='fullName',
    how='left'
)
df_active_filtered

,fullName,highest_ELO,game_date,ELO,min,max
0,Myles Turner,1384.659858,1946-11-25,1000.000000,2015-10-03,2025-06-22
1,Myles Turner,1384.659858,1946-11-26,1000.000000,2015-10-03,2025-06-22
2,Myles Turner,1384.659858,1946-11-30,1000.000000,2015-10-03,2025-06-22
3,Myles Turner,1384.659858,1946-12-05,1000.000000,2015-10-03,2025-06-22
4,Myles Turner,1384.659858,1946-12-07,1000.000000,2015-10-03,2025-06-22
...,...,...,...,...,...,...
4086588,George Mikan,1304.147486,2025-06-11,1282.795065,1948-11-09,1956-03-21
4086589,George Mikan,1304.147486,2025-06-13,1282.795065,1948-11-09,1956-03-21
4086590,George Mikan,1304.147486,2025-06-16,1282.795065,1948-11-09,1956-03-21
4086591,George Mikan,1304.147486,2025-06-19,1282.795065,1948-11-09,1956-03-21


In [ ]:
elo_active_filtered_df = df_active_filtered[(df_active_filtered['min']<=df_active_filtered['game_date'])&(df_active_filtered['game_date']<=df_active_filtered['max'])][['game_date','fullName','ELO']]
elo_active_filtered_df

,game_date,fullName,ELO
12011,2015-10-03,Myles Turner,996.503825
12012,2015-10-04,Myles Turner,996.503825
12013,2015-10-05,Myles Turner,996.503825
12014,2015-10-06,Myles Turner,999.592828
12015,2015-10-07,Myles Turner,999.592828
...,...,...,...
4073304,1956-03-15,George Mikan,1290.933376
4073305,1956-03-16,George Mikan,1295.858039
4073306,1956-03-17,George Mikan,1284.074152
4073307,1956-03-19,George Mikan,1294.343228


In [ ]:
df_active_players_pivot = elo_active_filtered_df.pivot(index='game_date', columns='fullName', values='ELO')
df_active_players_pivot

fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1948-11-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-11-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-11-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-11-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_active_players_pivot.index = pd.to_datetime(df_active_players_pivot.index)
df_active_players_quarter_avg = df_active_players_pivot.resample('Q').mean()
df_active_players_month_avg = df_active_players_pivot.resample('M').mean()
df_active_players_year_avg = df_active_players_pivot.resample('Y').mean()
df_active_players_quarter_avg.dropna(how='all', inplace=True)
df_active_players_month_avg.dropna(how='all', inplace=True)
df_active_players_year_avg.dropna(how='all', inplace=True)
df_active_players_quarter_avg

/tmp/ipython-input-2875427888.py:2: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_active_players_quarter_avg = df_active_players_pivot.resample('Q').mean()
/tmp/ipython-input-2875427888.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_active_players_month_avg = df_active_players_pivot.resample('M').mean()


fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1948-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,1262.491393,1449.034132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30,1258.100367,1487.088443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-12-31,1214.592007,1528.746475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
bcr.bar_chart_race(
    df=df_active_players_month_avg.fillna(0),
    filename='race_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.mp4',           # or None to show in notebook
    orientation='h',               # horizontal bars
    sort='desc',
    n_bars=15,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=20,
    period_length=1000,
    title='Monthly ELO Rating - Active NBA players',
    interpolate_period=False,
    bar_size=.8,
    cmap='dark12',
    dpi=144,
    period_label={'x': 0.99, 'y': 0.1, 'ha': 'right', 'va': 'center'},
)

/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 127 () missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 128 (\x80) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 129 (\x81) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 130 (\x82) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local

In [ ]:
bcr.bar_chart_race(
    df=df_active_players_year_avg.fillna(0),
    filename='race_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.mp4',           # or None to show in notebook
    orientation='h',               # horizontal bars
    sort='desc',
    n_bars=15,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=20,
    period_length=1000,
    title='Yearly ELO Rating - Active NBA players',
    interpolate_period=False,
    bar_size=.8,
    cmap='dark12',
    dpi=144,
    period_label={'x': 0.99, 'y': 0.1, 'ha': 'right', 'va': 'center'},
)

# Last ELO rating of NBA players over time

In [ ]:
elo_history_filtered_df = df_filtered[['game_date','fullName','ELO']]
elo_history_filtered_df

,game_date,fullName,ELO
0,1946-11-25,Myles Turner,1000.000000
1,1946-11-26,Myles Turner,1000.000000
2,1946-11-30,Myles Turner,1000.000000
3,1946-12-05,Myles Turner,1000.000000
4,1946-12-07,Myles Turner,1000.000000
...,...,...,...
4086588,2025-06-11,George Mikan,1282.795065
4086589,2025-06-13,George Mikan,1282.795065
4086590,2025-06-16,George Mikan,1282.795065
4086591,2025-06-19,George Mikan,1282.795065


In [ ]:
df_history_players_pivot = elo_history_filtered_df.pivot(index='game_date', columns='fullName', values='ELO')
df_history_players_pivot

fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1946-11-25,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-11-26,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-11-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-12-05,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-12-07,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-11,1249.239956,1470.117638,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477
2025-06-13,1249.239956,1470.117638,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477
2025-06-16,1249.239956,1470.117638,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477


In [ ]:
df_history_players_pivot.index = pd.to_datetime(df_history_players_pivot.index)
df_history_players_quarter_avg = df_history_players_pivot.resample('Q').mean()
df_history_players_month_avg = df_history_players_pivot.resample('M').mean()
df_history_players_year_avg = df_history_players_pivot.resample('Y').mean()
df_history_players_quarter_avg.dropna(how='all', inplace=True)
df_history_players_month_avg.dropna(how='all', inplace=True)
df_history_players_year_avg.dropna(how='all', inplace=True)
df_history_players_quarter_avg

/tmp/ipython-input-4290293294.py:2: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_history_players_quarter_avg = df_history_players_pivot.resample('Q').mean()
/tmp/ipython-input-4290293294.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_history_players_month_avg = df_history_players_pivot.resample('M').mean()


fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1946-12-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1947-03-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1947-06-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1947-12-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1948-03-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,1262.491393,1449.034132,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477
2024-06-30,1258.100367,1487.088443,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477
2024-12-31,1214.592007,1528.746475,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477


In [ ]:
bcr.bar_chart_race(
    df=df_history_players_month_avg.fillna(0),
    filename='race_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.mp4',           # or None to show in notebook
    orientation='h',               # horizontal bars
    sort='desc',
    n_bars=15,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=20,
    period_length=1000,
    title='Monthly ELO Rating - Historic NBA players',
    interpolate_period=False,
    bar_size=.8,
    cmap='dark12',
    dpi=144,
    period_label={'x': 0.99, 'y': 0.1, 'ha': 'right', 'va': 'center'},
)

/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 127 () missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 128 (\x80) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 129 (\x81) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 130 (\x82) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local

In [ ]:
bcr.bar_chart_race(
    df=df_history_players_year_avg.fillna(0),
    filename='race_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.mp4',           # or None to show in notebook
    orientation='h',               # horizontal bars
    sort='desc',
    n_bars=15,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=20,
    period_length=1000,
    title='Yearly ELO Rating - Historic NBA players',
    interpolate_period=False,
    bar_size=.8,
    cmap='dark12',
    dpi=144,
    period_label={'x': 0.99, 'y': 0.1, 'ha': 'right', 'va': 'center'},
)

# Max ELO rating of NBA players over time

In [ ]:
display(df_history_players_pivot)
elo_max_df = df_history_players_pivot.cummax()
display(elo_max_df)

fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1946-11-25,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-11-26,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-11-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-12-05,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
1946-12-07,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-11,1249.239956,1470.117638,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477
2025-06-13,1249.239956,1470.117638,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477
2025-06-16,1249.239956,1470.117638,1282.694559,1071.328057,1053.673988,1215.873485,1339.538782,1027.632977,1142.85202,1535.691412,...,1138.284188,875.952712,1129.721769,1304.304568,1482.86635,1196.937081,1213.775383,1479.406986,1242.570098,1348.11477


fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1946-11-25,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1946-11-26,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1946-11-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1946-12-05,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1946-12-07,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-11,1332.244776,1544.795397,1366.398802,1306.306475,1384.505409,1528.950818,1368.71409,1357.548467,1376.088796,1700.034146,...,1466.427282,1312.215989,1386.347495,1345.942337,1623.868533,1326.075231,1320.714196,1510.867878,1517.550312,1369.595088
2025-06-13,1332.244776,1544.795397,1366.398802,1306.306475,1384.505409,1528.950818,1368.71409,1357.548467,1376.088796,1700.034146,...,1466.427282,1312.215989,1386.347495,1345.942337,1623.868533,1326.075231,1320.714196,1510.867878,1517.550312,1369.595088
2025-06-16,1332.244776,1544.795397,1366.398802,1306.306475,1384.505409,1528.950818,1368.71409,1357.548467,1376.088796,1700.034146,...,1466.427282,1312.215989,1386.347495,1345.942337,1623.868533,1326.075231,1320.714196,1510.867878,1517.550312,1369.595088


In [ ]:
elo_max_df.index = pd.to_datetime(elo_max_df.index)
elo_max_df_quarter = elo_max_df.resample('Q').max()
elo_max_df_month = elo_max_df.resample('M').max()
elo_max_df_year = elo_max_df.resample('Y').max()
elo_max_df_quarter.dropna(how='all', inplace=True)
elo_max_df_month.dropna(how='all', inplace=True)
elo_max_df_year.dropna(how='all', inplace=True)
elo_max_df_month

/tmp/ipython-input-2778779465.py:2: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  elo_max_df_quarter_avg = elo_max_df.resample('Q').max()
/tmp/ipython-input-2778779465.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  elo_max_df_month_avg = elo_max_df.resample('M').max()


fullName,Aaron Gordon,Al Horford,Al-Farouq Aminu,Alex English,Allen Iverson,Alonzo Mourning,Alton Lister,Amar'e Stoudemire,Anderson Varejao,Andre Iguodala,...,Walt Frazier,Walter Davis,Wes Unseld,Willis Reed,Wilt Chamberlain,World Free,Xavier McDaniel,Yao Ming,Zach Randolph,Zydrunas Ilgauskas
game_date,,,,,,,,,,,,,,,,,,,,,
1946-11-30,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1946-12-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1947-01-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1947-02-28,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
1947-03-31,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,1332.244776,1544.795397,1366.398802,1306.306475,1384.505409,1528.950818,1368.71409,1357.548467,1376.088796,1700.034146,...,1466.427282,1312.215989,1386.347495,1345.942337,1623.868533,1326.075231,1320.714196,1510.867878,1517.550312,1369.595088
2025-03-31,1332.244776,1544.795397,1366.398802,1306.306475,1384.505409,1528.950818,1368.71409,1357.548467,1376.088796,1700.034146,...,1466.427282,1312.215989,1386.347495,1345.942337,1623.868533,1326.075231,1320.714196,1510.867878,1517.550312,1369.595088
2025-04-30,1332.244776,1544.795397,1366.398802,1306.306475,1384.505409,1528.950818,1368.71409,1357.548467,1376.088796,1700.034146,...,1466.427282,1312.215989,1386.347495,1345.942337,1623.868533,1326.075231,1320.714196,1510.867878,1517.550312,1369.595088


In [ ]:
bcr.bar_chart_race(
    df=elo_max_df_month.fillna(0),
    filename='race_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.mp4',           # or None to show in notebook
    orientation='h',               # horizontal bars
    sort='desc',
    n_bars=15,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=20,
    period_length=1000,
    title='Max ELO Rating (Monthly) - Historic NBA players',
    interpolate_period=False,
    bar_size=.8,
    cmap='dark12',
    dpi=144,
    period_label={'x': 0.99, 'y': 0.1, 'ha': 'right', 'va': 'center'},
)

/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 127 () missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 128 (\x80) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 129 (\x81) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local/lib/python3.12/dist-packages/bar_chart_race/_make_chart.py:284: UserWarning: Glyph 130 (\x82) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(io.BytesIO())
/usr/local

Exception: You do not have ffmpeg installed on your machine. Download
                            ffmpeg from here: https://www.ffmpeg.org/download.html.
                            
                            Matplotlib's original error message below:

                            Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '1020x504', '-pix_fmt', 'rgba', '-framerate', '20.0', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', 'race_2025_08_19_20_34_10.mp4']' returned non-zero exit status 255.
                            

In [ ]:
bcr.bar_chart_race(
    df=elo_max_df_month.fillna(0),
    filename='race_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.mp4',           # or None to show in notebook
    orientation='h',               # horizontal bars
    sort='desc',
    n_bars=15,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=20,
    period_length=1000,
    title='Max ELO Rating (Yearly) - Historic NBA players',
    interpolate_period=False,
    bar_size=.8,
    cmap='dark12',
    dpi=144,
    period_label={'x': 0.99, 'y': 0.1, 'ha': 'right', 'va': 'center'},
)